## Simple Collision avoidance

In [18]:
import os
import subprocess
import numpy as np

# Custom modules
from vehicle_logic import VehicleLogic,VehicleMode
from helpers.change_coordinates import heading_to_yaw,find_spawns,global2local
from helpers.visualization.gazebo_world import update_world
from helpers.visualization.plots import plot_3d_interactive
from plan import Plan,State
from gcs import GCS

Kill all related process

In [19]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

Paths

In [20]:
QGC_path='~/QGroundControl.AppImage'
gazebo_word_path="gazebo_worlds/runway.world"
ardupilot_vehicle_path='~/ardupilot/Tools/autotest/sim_vehicle.py'

## Choose Simulator

In [21]:
simulator ='gazebo' # 'QGroundControl' #

## Create Plans

In [22]:
offsets = [(-10, 0, 0, 0),(10, 0, 0, 0)]#
n_uavs = len(offsets)


homes=np.array([offset[:3] for offset in offsets])

gcs=GCS(positions=homes.copy())
global_paths=[np.array([[-10, 0, 5],[10, 0, 5]]),
              np.array([[10, 0, 5],[-10, 0, 5]])]

local_paths=[global2local(path, home,pairwise=True) for path,home in zip(global_paths,homes)]


radar_radius=10
avoidance_radius =5


## Normal behavior
markers = {f'waypoints': {'pos':global_paths[0],'color':'blue'},
       #f'real pos': {'pos':global_paths[1],'color':'red','radius':0.1},
       #f'avoid zone': {'pos':global_paths[1],'color':'orange','radius':avoidance_radius,'alpha': 0.84},
       #f'radar zone': {'pos':global_paths[1],'color':'orange','radius':radar_radius,'alpha': 0.9}
       }




## Remote ID attack
# fake_posistion=np.array([[0,0,5]])
# markers = {f'waypoints': {'pos':global_paths[0],'color':'blue'},
#        f'real pos': {'pos':global_paths[1],'color':'red','radius':0.1},
#        # f'fake_pos': {'pos':fake_posistion,'color':'orange','radius':0.1},
#        # f'avoid zone': {'pos':fake_posistion,'color':'red','radius':avoidance_radius,'alpha': 0.85},
#        # f'radar zone': {'pos':global_paths[1],'color':'orange','radius':radar_radius,'alpha': 0.9}
#        }




plans=[Plan.basic(wps=local_paths[0],alt=5,wp_margin=1,navegation_speed=3,verbose=1),
       Plan.basic(wps=local_paths[1],alt=5,wp_margin=1,navegation_speed=3,verbose=1)]
print(plans)


markers_plot = {f'waypoints': {'pos':global_paths[0],'color':'green'},
                #f'uav start': {'pos':np.array([[-10, 0, 0]]),'color':'green'},
                #'uav goal': {'pos':np.array([[10, 0, 0]]),'color':'green'},
              #f'adversary pos': {'pos':global_paths[1],'color':'red','radius':0.1},
              #f'adversary fake pos': {'pos':fake_posistion,'color':'orange','radius':0.1}
              }


plot_3d_interactive(markers_plot,title='Simulation Markers',expand=[0.4,0.4,0.2],ground=-0.05)
#fake_posistion=fake_posistion[0]

if simulator == 'QGroundControl':
    home_position=(-35.3633245,149.1652241,0,0)
    spawns=find_spawns(home_position, offsets)

Plan 'basic' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created — no connection yet 🧩
Action 'Set Navigation Speed' created — no connection yet 🧩
Step 'Set speed to 3.00 m/s' created — no connection yet 🧩
Action 'Arm' created — no connection yet 🧩
Step 'arm' created — no connection yet 🧩
Action 'takeoff' created — no connection yet 🧩
Step 'takeoff' created — no connection yet 🧩
Action 'fly' created — no connection yet 🧩
Step 'go to  -> (0, 0, 5)' created — no connection yet 🧩
Step 'go to  -> (20, 0, 5)' created — no connection yet 🧩
Action 'Land' created — no connection yet 🧩
Step 'land' created — no connection yet 🧩
Plan 'basic' created — no connection yet 🧩
Action 'Pre-Arm 

## Waypoint seletion algorithm

In [23]:
if simulator == 'QGroundControl':
    sim_path = os.path.expanduser(QGC_path)
    sim_cmd =[sim_path]
elif simulator == 'gazebo':
    # Convert to Gazebo format (name, x, y, z, roll, pitch, yaw)
    drones = [(east, north, up, 0, 0, heading_to_yaw(heading)) for i, (east, north, up, heading) in enumerate(offsets)]
    world_path = os.path.expanduser(gazebo_word_path)
    updated_world_path = update_world(drones,markers,world_path)
    sim_cmd = ["gazebo", "--verbose", updated_world_path] 

simulator_process = subprocess.Popen(
                    sim_cmd,
                    stdout=subprocess.DEVNULL,  # Suppress standard output
                    stderr=subprocess.DEVNULL,  # Suppress error output
                    shell=False  # Ensure safety when passing arguments
                    )

Launch Vehicle

In [24]:
vehicle_path = os.path.expanduser(ardupilot_vehicle_path)

In [25]:
for i in range(n_uavs):
    vehicle_cmd = f"python3 {vehicle_path} -v ArduCopter -I{i} --sysid {i+1} --no-rebuild"
    if simulator == 'QGroundControl':
        spawn=','.join(map(str, spawns[i]))
        vehicle_cmd += f" --custom-location={spawn}"
    elif simulator == 'gazebo':
        vehicle_cmd += " -f gazebo-iris"
    subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

Create UavLogic

In [26]:
uavs=[]
for i,(plan,home) in enumerate(zip(plans,homes)):
    uavs.append(VehicleLogic(sys_id=i+1,
                    home=home,
                    plan= plan))

Vehicle 1: Step 'Check disarmed' is now connected ✅🔗
Vehicle 1: Step 'Check EKF' is now connected ✅🔗
Vehicle 1: Step 'Check GPS' is now connected ✅🔗
Vehicle 1: Step 'Check system' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Step 'Switch to GUIDED' is now connected ✅🔗
Vehicle 1: Action 'Set Mode: GUIDED' is now connected ✅🔗
Vehicle 1: Action 'Set Mode: GUIDED' is now connected ✅🔗
Vehicle 1: Step 'Set speed to 3.00 m/s' is now connected ✅🔗
Vehicle 1: Action 'Set Navigation Speed' is now connected ✅🔗
Vehicle 1: Action 'Set Navigation Speed' is now connected ✅🔗
Vehicle 1: Step 'arm' is now connected ✅🔗
Vehicle 1: Action 'Arm' is now connected ✅🔗
Vehicle 1: Action 'Arm' is now connected ✅🔗
Vehicle 1: Step 'takeoff' is now connected ✅🔗
Vehicle 1: Action 'takeoff' is now connected ✅🔗
Vehicle 1: Action 'takeoff' is now connected ✅🔗
Vehicle 1: Step 'go to  -> (0, 0, 5)' is now connected ✅🔗
Vehicle 1: 

In [27]:
plans[0]

🚀 <Plan 'basic'>
  🕓 <Action 'Pre-Arm Check'>
    🕓 <Step 'Check disarmed'>
    🕓 <Step 'Check EKF'>
    🕓 <Step 'Check GPS'>
    🕓 <Step 'Check system'>
  🕓 <Action 'Set Mode: GUIDED'>
    🕓 <Step 'Switch to GUIDED'>
  🕓 <Action 'Set Navigation Speed'>
    🕓 <Step 'Set speed to 3.00 m/s'>
  🕓 <Action 'Arm'>
    🕓 <Step 'arm'>
  🕓 <Action 'takeoff'>
    🕓 <Step 'takeoff'>
  🕓 <Action 'fly'>
    🕓 <Step 'go to  -> (0, 0, 5)'>
    🕓 <Step 'go to  -> (20, 0, 5)'>
  🕓 <Action 'Land'>
    🕓 <Step 'land'>

In [28]:
plans[1]

🚀 <Plan 'basic'>
  🕓 <Action 'Pre-Arm Check'>
    🕓 <Step 'Check disarmed'>
    🕓 <Step 'Check EKF'>
    🕓 <Step 'Check GPS'>
    🕓 <Step 'Check system'>
  🕓 <Action 'Set Mode: GUIDED'>
    🕓 <Step 'Switch to GUIDED'>
  🕓 <Action 'Set Navigation Speed'>
    🕓 <Step 'Set speed to 3.00 m/s'>
  🕓 <Action 'Arm'>
    🕓 <Step 'arm'>
  🕓 <Action 'takeoff'>
    🕓 <Step 'takeoff'>
  🕓 <Action 'fly'>
    🕓 <Step 'go to  -> (0, 0, 5)'>
    🕓 <Step 'go to  -> (-20, 0, 5)'>
  🕓 <Action 'Land'>
    🕓 <Step 'land'>

## Execution

In [29]:
while gcs.incomplete_missions:
    ## Get global positions
    to_remove = set()
    for id in gcs.incomplete_missions:
        uav=uavs[id-1]
        if uav.current_action().name =='fly':
            gcs.ask_position(uav) 
            #gcs.poss[1] = fake_posistion  
    for id in gcs.incomplete_missions:
        uav = uavs[id-1]
        if uav.current_action().name =='fly':
            jd,min_dist=gcs.get_nearest_neighbor(uav)
            if min_dist<avoidance_radius and uav.get_mode()==VehicleMode.MISSION:  
                gcs.send_neighborg_position(uav,neighborg_id=jd)
                uav.set_mode(VehicleMode.AVOIDANCE)
            if uav.get_mode()==VehicleMode.AVOIDANCE and uav.current_step().state==State.DONE:
                uav.set_mode(VehicleMode.MISSION)
        uav.plan.act()

        if uav.plan.state == State.DONE:
            to_remove.add(id)
    gcs.update_missions(to_remove)
    

Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 2: ▶️ Starting Step: Check disarmed
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 2: ✅ Step: Check disarmed is done
Vehicle 2: ▶️ Starting Step: Check EKF
Vehicle 2: ✅ Step: Check EKF is done
Vehicle 2: ▶️ Starting Step: Check GPS
Vehicle 1: ✅ Step: Check EKF is done
Vehicle 1: ▶️ Starting Step: Check GPS
Vehicle 2: ✅ Step: Check GPS is done
Vehicle 2: ▶️ Starting Step: Check system
Vehicle 1: ✅ Step: Check GPS is done
Vehicle 1: ▶️ Starting Step: Check system
Vehicle 2: ✅ Step: Check system is done
Vehicle 2: ▶️ Starting Step: Switch to GUIDED
Vehicle 2: ✅ Step: Switch to GUIDED is done
Vehicle 2: ▶️ Starting Step: Set speed to 3.00 m/s
Vehicle 2: ✅ Step: Set speed to 3.00 m/s is done
Vehicle 2: ▶️ Starting Step: arm
Vehicle 2: ✅ Step: arm is done
Vehicle 2: ▶️ Starting Step: takeoff
Vehicle 2:📍 Distance to target: 5.04 m
Vehicle 2:📍 Distance to target: 5.04 m
Vehicle 2:📍 Distance to target:

In [30]:
uavs[0].plan

✅ <Plan 'basic'>
  ✅ <Action 'Pre-Arm Check'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'Set Mode: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'Set Navigation Speed'>
    ✅ <Step 'Set speed to 3.00 m/s'>
  ✅ <Action 'Arm'>
    ✅ <Step 'arm'>
  ✅ <Action 'takeoff'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'fly'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
    ✅ <Step 'go to (avoidance) -> (7.118408146816505, 0.989441512847776, 4.741828441619873)'>
    ✅ <Step 'go to (avoidance) -> (7.944764055512881, 1.7702670635995759, 4.708235263824463)'>
    ✅ <Step 'go to (avoidance) -> (8.358284383369739, 2.3376891785577163, 4.674014091491699)'>
    ✅ <Step 'go to (avoidance) -> (9.094539543882995, 2.3908821783311636, 4.6610426902771)'>
    ✅ <Step 'go to (avoidance) -> (9.593281631043276, 2.523156367514336, 4.635769367218018)'>
    ✅ <Step 'go to (avoidance) -> (11.22781761160492, 2.26272921115426, 4.676211833953857)'>
    ✅ <

In [31]:
uavs[1].plan

✅ <Plan 'basic'>
  ✅ <Action 'Pre-Arm Check'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'Set Mode: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'Set Navigation Speed'>
    ✅ <Step 'Set speed to 3.00 m/s'>
  ✅ <Action 'Arm'>
    ✅ <Step 'arm'>
  ✅ <Action 'takeoff'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'fly'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
    ✅ <Step 'go to (avoidance) -> (-7.893691960293068, -1.0002068713041865, 4.781832695007324)'>
    ✅ <Step 'go to (avoidance) -> (-8.690851973692094, -1.7216783547171453, 4.747145652770996)'>
    ✅ <Step 'go to (avoidance) -> (-9.17488661344176, -2.446557181449257, 4.732285499572754)'>
    ✅ <Step 'go to (avoidance) -> (-10.61646782381399, -2.679593303437896, 4.7152581214904785)'>
    ✅ <Step 'go to (avoidance) -> (-11.115973689142956, -2.557417482383767, 4.70418119430542)'>
    ✅ <Step 'go to  -> (-20, 0, 5)'>
  ✅ <Action 'Land'>
    ✅ <Step 'land'>

## Kill all related process

In [32]:
# for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
#     os.system(f"pkill -9 -f {process}")